## Loading the data into Starfish

In [4]:
from starfish.io import Stack
from starfish.image import ImageStack
from starfish.constants import Indices
import os

This notebook demonstrates how to load osmFISH data into starfish. Below loads fov_001, however fovs 002 and 003 are also converted and can be loaded by exchanging the number in the cloudflare link. The data can be dumped for local loading with `s.image.write`

In [2]:
s = Stack.from_experiment_json('https://dmf0bdeheu4zf.cloudfront.net/20180722/osmFISH/fov_001/experiment.json')

The below plot displays the z-volume for channel 0 of fov_001

In [3]:
s.image.show_stack({Indices.CH: 0});

interactive(children=(IntSlider(value=0, description='plane_index', max=38), Output()), _dom_classes=('widget-interact',))